# **Phase 2 – Data Preparation (ETL)**

## Overview
Phase 2 focuses on **cleaning, transforming, and structuring** the raw job offer data collected in Phase 1.

We'll:
1. **Load** the raw CSV from Phase 1
2. **Clean** duplicates and handle missing values
3. **Standardize** salary formats to monthly EUR values
4. **Parse** French relative dates (aujourd'hui, hier, il y a X jours)
5. **Process** job descriptions with NLTK (remove stopwords, punctuation)
6. **Extract** keywords using TF-IDF vectorization
7. **Encode** categorical variables (sectors, locations, contract types)
8. **Save** the cleaned dataset for Phase 3 (ML)

### **Phase Outcome**
✅ Clean, structured, ML-ready dataset saved to `hellowork_cleaned.csv`

## **Step 1 – Import Libraries**

Below are all the libraries we need for ETL:

- **pandas/numpy**: Load, clean, and manipulate datasets
- **re, string**: Text pattern matching and punctuation removal  
- **nltk**: French text preprocessing (stopwords, tokenization)
- **sklearn.feature_extraction**: TF-IDF vectorization for text
- **sklearn.preprocessing**: Encode categorical variables (OneHot, Label)
- **datetime**: Parse and manipulate publication dates
- **Path, os**: File system operations

In [1]:
# --- Import Libraries ---
import pandas as pd
import numpy as np
from pathlib import Path
import os
import re
import string
from datetime import datetime, timedelta

# NLP and Text Processing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

# Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Setup French stopwords
FRENCH_STOPWORDS = set(stopwords.words('french'))

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## **Step 2 – Load Raw Dataset**

Load the final scraped CSV from Phase 1 (`hellowork_final_sectors_data.csv`).

**Objectives:**
- Read the CSV file with UTF-8 encoding
- Display shape, columns, and preview data
- Understand the raw data structure before processing

In [2]:
# --- Step 2: Load Raw Dataset ---
raw_path = "data/raw/hellowork_final_sectors_data.csv"

df = pd.read_csv(raw_path, encoding='utf-8-sig')

print(f"📊 Dataset shape: {df.shape}")
print(f"\n📋 Columns:\n{df.columns.tolist()}")
print(f"\n🔍 Data types:\n{df.dtypes}")
print(f"\n📈 First 3 rows:\n")
df.head(3)

📊 Dataset shape: (1374, 9)

📋 Columns:
['Sector', 'Job_Title', 'Company', 'Location', 'Contract', 'Salary', 'Description', 'Publication_Date', 'URL']

🔍 Data types:
Sector               object
Job_Title            object
Company              object
Location             object
Contract             object
Salary               object
Description          object
Publication_Date    float64
URL                  object
dtype: object

📈 First 3 rows:



,Sector,Job_Title,Company,Location,Contract,Salary,Description,Publication_Date,URL
0,Agriculture • Pêche,Alternance - Chargé·e de Formation H/F,Remy Cointreau,Paris - 75,Alternance,"486,49 - 1 801,80 € / mois",Nous recherchons un·e candidat·e : Alternance...,NaN,https://www.hellowork.com/fr-fr/emplois/642118...
1,BTP,Alternance-Gestionnaire Paie H/F,Lafarge France,Issy-les-Moulineaux - 92,Alternance,"486,49 - 1 801,80 € / mois",Pourquoi nous rejoindre ? > Participer à la t...,NaN,https://www.hellowork.com/fr-fr/emplois/729761...
2,BTP,Ouvrier Polyvalent en Menuiserie H/F,Groupe Actual,Auterive - 31,Intérim,"Estimation → 12,36 - 13,50 € / heure",Nous recherchons un(e) menuisier(e) expériment...,NaN,https://www.hellowork.com/fr-fr/emplois/735245...


## **Step 3 – Clean Duplicates & Handle Missing Values**

Before processing, we need to:
1. **Remove duplicate rows** (same job posted multiple times)
2. **Identify missing values** in key columns
3. **Fill text columns** with `"Not specified"` where empty
4. **Verify data consistency**

In [3]:
# --- Step 3: Clean Duplicates & Handle Missing Values ---

# Remove exact duplicates
before_clean = len(df)
df = df.drop_duplicates()
print(f"🗑️  Removed duplicates: {before_clean} → {len(df)} rows")

# Handle missing values in text columns
text_columns = ["Job_Title", "Company", "Location", "Contract", "Description"]
for col in text_columns:
    if col in df.columns:
        missing_count = df[col].isna().sum()
        df[col] = df[col].fillna("Not specified")
        print(f"   {col}: filled {missing_count} missing values")

# Show missing values after cleaning
print(f"\n✅ Missing values after cleaning:")
print(df.isna().sum())
print(f"\n📊 Current shape: {df.shape}")

🗑️  Removed duplicates: 1374 → 1219 rows
   Job_Title: filled 0 missing values
   Company: filled 15 missing values
   Location: filled 0 missing values
   Contract: filled 0 missing values
   Description: filled 0 missing values

✅ Missing values after cleaning:
Sector                 0
Job_Title              0
Company                0
Location               0
Contract               0
Salary                 0
Description            0
Publication_Date    1219
URL                    0
dtype: int64

📊 Current shape: (1219, 9)


## **Step 4 – Parse French Relative Dates**

Job postings use French relative dates like:
- `"aujourd'hui"` (today)
- `"hier"` (yesterday)  
- `"il y a 3 jours"` (3 days ago)
- `"il y a 2 mois"` (2 months ago)

We need to **convert these to actual timestamps** for analysis and temporal trends.

In [4]:
# --- Step 4: Parse French Relative Dates ---

def parse_relative_date(date_str):
    """Convert French relative dates to timestamps.
    
    Examples:
    - "aujourd'hui" → today's date
    - "hier" → yesterday's date
    - "il y a 3 jours" → 3 days ago
    - "il y a 2 mois" → ~60 days ago
    """
    if pd.isna(date_str) or date_str == "Not specified":
        return pd.NaT
    
    date_str = str(date_str).lower()
    today = datetime.today().date()
    
    try:
        if "hier" in date_str:
            return pd.Timestamp(today - timedelta(days=1))
        elif "aujourd'hui" in date_str or "today" in date_str:
            return pd.Timestamp(today)
        elif "il y a" in date_str:
            # Extract the number (e.g., "il y a 3 jours" → 3)
            nums = re.findall(r'\d+', date_str)
            if nums:
                val = int(nums[0])
                if "mois" in date_str:
                    return pd.Timestamp(today - timedelta(days=val*30))
                elif "jour" in date_str:
                    return pd.Timestamp(today - timedelta(days=val))
                elif "heure" in date_str or "minute" in date_str:
                    return pd.Timestamp(today)  # Same day
        # Fallback: try standard date parsing
        return pd.to_datetime(date_str, dayfirst=True, errors='coerce')
    except Exception:
        return pd.NaT


# Apply date parsing
if "Publication_Date" in df.columns:
    df["Publication_Date"] = df["Publication_Date"].apply(parse_relative_date)
    print(f"📅 Date range: {df['Publication_Date'].min()} to {df['Publication_Date'].max()}")
    print(f"   Missing dates: {df['Publication_Date'].isna().sum()}")
else:
    print("⚠️  'Publication_Date' column not found")

📅 Date range: NaT to NaT
   Missing dates: 1219


## **Step 5 – Normalize Salary Fields**

Salaries come in **many formats**:
- `"30k€"`, `"30 000 €"` (monthly or annual?)
- `"2500 €/mois"` (monthly)
- `"36000 €/an"` (annual)
- `"2500-3000"` (range)

We'll **extract numeric values** and **convert to monthly EUR**, handling ranges by averaging.

In [5]:
# --- Step 5: Normalize Salary Fields ---

def normalize_salary(value):
    """Convert salary strings to monthly numeric value (EUR).
    
    Handles:
    - Ranges: "2500-3000" → 2750 (average)
    - Annual: "36000 €/an" → 3000 (monthly)
    - Hourly: "15 €/h" → 2400 (assuming 160h/month)
    - Missing: returns NaN
    """
    if not isinstance(value, str) or not value.strip():
        return np.nan
    
    s = value.lower()
    # Clean whitespace variants
    s = s.replace("\u202f", " ").replace("\xa0", " ").replace("\u2009", " ")
    s = s.replace("à partir de", "").replace("estimation", "").strip()
    
    # Detect units
    per_hour = "heure" in s or "/heure" in s or "€/h" in s
    per_year = "an" in s or "/an" in s or "annuel" in s
    per_month = "mois" in s or "/mois" in s or "mensuel" in s
    
    # Extract all numbers
    num_matches = re.findall(r"\d+[\d\.\s]*[\,\.]?\d*", s)
    clean_nums = []
    for m in num_matches:
        m_clean = m.strip().replace(" ", "").replace("\u202f", "")
        if m_clean.count(',') > 0 and m_clean.count('.') == 0:
            m_clean = m_clean.replace(',', '.')
        if m_clean.count('.') > 1:
            m_clean = m_clean.replace('.', '')
        m_clean = re.sub(r"[^0-9\.]", "", m_clean)
        try:
            clean_nums.append(float(m_clean))
        except:
            continue
    
    if not clean_nums:
        return np.nan
    
    # Average if range
    val = float(sum(clean_nums[:2]) / len(clean_nums[:2])) if len(clean_nums) > 1 else float(clean_nums[0])
    
    # Detect 'k' multiplier
    if 'k' in s:
        val = val * 1000
    
    # Convert to monthly
    if per_hour:
        monthly = val * 160  # 160 working hours/month
    elif per_year:
        monthly = val / 12
    elif per_month:
        monthly = val
    else:
        # Ambiguous: assume annual if > 5000, else monthly
        monthly = val / 12 if val > 5000 else val
    
    return float(monthly)


# Apply salary normalization
if "Salary" in df.columns:
    df["Salary_Monthly"] = df["Salary"].astype(str).apply(normalize_salary)
    print(f"💰 Salary statistics (monthly EUR):")
    print(df["Salary_Monthly"].describe())
    print(f"\n   Missing salaries: {df['Salary_Monthly'].isna().sum()}")
else:
    print("⚠️  'Salary' column not found")

💰 Salary statistics (monthly EUR):
count    1057.000000
mean     2104.548929
std       543.763565
min         1.004167
25%      1900.800000
50%      2116.900000
75%      2300.000000
max      8229.000000
Name: Salary_Monthly, dtype: float64

   Missing salaries: 162


## **Step 6 – Clean Text with NLTK**

Job descriptions contain:
- Punctuation, line breaks, excessive whitespace
- Common French words (stopwords) that don't add meaning
- Mixed case and special characters

We'll **preprocess text** for ML:
1. Convert to lowercase
2. Remove punctuation and line breaks
3. **Remove French stopwords** (le, la, de, et, etc.)
4. This creates clean, tokenized descriptions ready for vectorization

In [6]:
# --- Step 6: Clean Text with NLTK ---

def clean_text(text: str) -> str:
    """Lowercase, remove punctuation, and drop French stopwords."""
    if pd.isna(text) or text == "Not specified":
        return ""
    text = text.lower()
    text = text.replace("\n", " ").strip()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = [word for word in text.split() if word not in FRENCH_STOPWORDS]
    return " ".join(tokens)

# Apply cleaning to descriptions
if "Description" in df.columns:
    df["Description_Clean"] = df["Description"].apply(clean_text)
    print("🧹 Cleaned text sample:")
    print(df[["Description", "Description_Clean"]].head(3))
else:
    print("⚠️  'Description' column not found")

🧹 Cleaned text sample:
                                         Description  \
0  Nous recherchons un·e candidat·e :  Alternance...   
1  Pourquoi nous rejoindre ?  > Participer à la t...   
2  Nous recherchons un(e) menuisier(e) expériment...   

                                   Description_Clean  
0  recherchons un·e candidat·e alternance chargé·...  
1  pourquoi rejoindre participer transformation d...  
2  recherchons menuisiere expérimentée rejoindre ...  


## **Step 7 – Extract Keywords with TF-IDF**

We use **TF-IDF vectorization** to surface the top keywords from each description.

**Why TF-IDF?** It weights words by importance: common words get low weight, rare informative words get higher weight.

**Goal:** Add a `Top_Keywords` column with the top terms per job description.

## **Step 8 – Encode Categorical Variables**

To make categories ML-friendly, we **factorize** columns like `Sector`, `Location`, `Contract`, `Company` into numeric codes.

This keeps the dataset lightweight and ready for clustering/classification.

In [7]:
# --- Step 7: Extract Keywords with TF-IDF ---

def extract_keywords_tfidf(texts, top_k=10):
    """Return top-k keywords for each document using TF-IDF."""
    vect = TfidfVectorizer(max_features=1000, stop_words=None)
    X = vect.fit_transform(texts)
    feature_names = vect.get_feature_names_out()
    
    def top_terms(row):
        if row.nnz == 0:
            return ""
        scores = zip(row.indices, row.data)
        sorted_terms = sorted(scores, key=lambda x: -x[1])[:top_k]
        return ",".join(feature_names[i] for i, _ in sorted_terms)

    return [top_terms(X[i]) for i in range(X.shape[0])]


text_col = "Description_Clean" if "Description_Clean" in df.columns else "Description"
texts = df[text_col].fillna("").astype(str).tolist()
df["Top_Keywords"] = extract_keywords_tfidf(texts, top_k=8)

print("🔑 Sample keywords:")
print(df[[text_col, "Top_Keywords"]].head(3))

🔑 Sample keywords:
                                   Description_Clean  \
0  recherchons un·e candidat·e alternance chargé·...   
1  pourquoi rejoindre participer transformation d...   
2  recherchons menuisiere expérimentée rejoindre ...   

                                        Top_Keywords  
0  formation,formations,groupe,aider,création,pla...  
1  paie,groupe,processus,contact,démarche,gestion...  
2  recherchons,connaissance,candidats,dexpérience...  


## **Step 9 – Save Cleaned Dataset**

Finally, we save the cleaned dataset for Phase 3 modeling.

**Output:** `data/processed/hellowork_cleaned.csv`

In [8]:
# --- Step 8: Encode Categorical Variables ---

categorical_cols = [c for c in ["Sector", "Location", "Contract", "Company"] if c in df.columns]
for c in categorical_cols:
    df[c + "_enc"] = pd.factorize(df[c].astype(str))[0]
    print(f"🏷️ Encoded {c} → {c+'_enc'}")

print(f"\n📊 Encoded columns added: {categorical_cols}")
print(f"Current shape: {df.shape}")

🏷️ Encoded Sector → Sector_enc
🏷️ Encoded Location → Location_enc
🏷️ Encoded Contract → Contract_enc
🏷️ Encoded Company → Company_enc

📊 Encoded columns added: ['Sector', 'Location', 'Contract', 'Company']
Current shape: (1219, 16)


## **Step 9 – Save Cleaned Dataset**

Finally, we save the cleaned dataset for Phase 3 modeling.

**Output:** `data/processed/hellowork_cleaned.csv`

In [ ]:
# --- Step 9: Save Cleaned Dataset ---

output_path = "data/processed/hellowork_cleaned.csv"
Path("data/processed").mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False, encoding="utf-8")
print(f"💾 Saved cleaned dataset to {output_path} with shape {df.shape}")

💾 Saved cleaned dataset to data/processed/hellowork_cleaned.csv with shape (1219, 16)


: 

In [3]:
# --- Data Quality Report ---
import pandas as pd
import numpy as np

print("="*70)
print("📊 PHASE 2 QUALITY REPORT")
print("="*70)

# Load the final cleaned data to verify
cleaned_df = pd.read_csv("data/processed/hellowork_cleaned.csv", encoding='utf-8')

print(f"\n✅ Dataset shape: {cleaned_df.shape}")
print(f"   → Rows: {cleaned_df.shape[0]} | Columns: {cleaned_df.shape[1]}")

print(f"\n📋 Column completeness:")
for col in cleaned_df.columns:
    non_null = cleaned_df[col].notna().sum()
    pct = (non_null / len(cleaned_df)) * 100
    print(f"   {col:25s}: {non_null:5d}/{len(cleaned_df):5d} ({pct:5.1f}%)")

print(f"\n💰 Salary Statistics (Salary_Monthly in EUR):")
salary_col = "Salary_Monthly" if "Salary_Monthly" in cleaned_df.columns else "salary_monthly"
salary_valid = cleaned_df[salary_col].dropna()
print(f"   Valid salaries: {len(salary_valid)}/{len(cleaned_df)} ({len(salary_valid)/len(cleaned_df)*100:.1f}%)")
if len(salary_valid) > 0:
    print(f"   Mean: €{salary_valid.mean():.2f}")
    print(f"   Median: €{salary_valid.median():.2f}")
    print(f"   Min: €{salary_valid.min():.2f}")
    print(f"   Max: €{salary_valid.max():.2f}")

print(f"\n📍 Geographic Distribution:")
if "Location" in cleaned_df.columns:
    top_locs = cleaned_df["Location"].value_counts().head(5)
    for loc, count in top_locs.items():
        print(f"   {loc:20s}: {count:4d} jobs")

print(f"\n🏢 Sector Distribution:")
if "Sector" in cleaned_df.columns:
    top_sectors = cleaned_df["Sector"].value_counts().head(5)
    for sector, count in top_sectors.items():
        print(f"   {sector:30s}: {count:4d} jobs")

print("\n" + "="*70)
print("✓ ETL PHASE COMPLETE - Data ready for ML (Phase 3)")
print("="*70)

📊 PHASE 2 QUALITY REPORT

✅ Dataset shape: (1219, 16)
   → Rows: 1219 | Columns: 16

📋 Column completeness:
   Sector                   :  1219/ 1219 (100.0%)
   Job_Title                :  1219/ 1219 (100.0%)
   Company                  :  1219/ 1219 (100.0%)
   Location                 :  1219/ 1219 (100.0%)
   Contract                 :  1219/ 1219 (100.0%)
   Salary                   :  1219/ 1219 (100.0%)
   Description              :  1219/ 1219 (100.0%)
   Publication_Date         :     0/ 1219 (  0.0%)
   URL                      :  1219/ 1219 (100.0%)
   Salary_Monthly           :  1057/ 1219 ( 86.7%)
   Description_Clean        :  1219/ 1219 (100.0%)
   Top_Keywords             :  1219/ 1219 (100.0%)
   Sector_enc               :  1219/ 1219 (100.0%)
   Location_enc             :  1219/ 1219 (100.0%)
   Contract_enc             :  1219/ 1219 (100.0%)
   Company_enc              :  1219/ 1219 (100.0%)

💰 Salary Statistics (Salary_Monthly in EUR):
   Valid salaries: 1057/1219 (

## **Étape Finale – Résumé et Visualisations des Données Nettoyées**

Vérifions la qualité et l'intégrité du dataset nettoyé avec des statistiques et visualisations.